# Titanic Survival Prediction

> ❗ Este modelo é fortemente inspirado [neste post da documentação do Danfo.](https://danfo.jsdata.org/examples/titanic-survival-prediction-using-danfo.js-and-tensorflow.js)
> 

### Importar *libs*

In [1]:
import dfd from 'danfojs-node';

const tf = dfd.tensorflow;

### Carregar dados

Os dados serão carregados de um `csv`, pré-baixado no diretório `./data` e que foi [obtido da Universidade de Stanford](https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv).

In [2]:


const load_data = async () => await dfd.readCSV( "./data/titanic.csv" );

let df = await load_data();

df.head().print(); // printa as primeiras 5 linhas do dataframe

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Name              │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ Mr. Owen Harris…  │ male              │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ Mrs. John Bradl…  │ female            │ 38                │ 1         

### Investigar os dados

Vamos operar alguns comandos para entender melhor os dados que temos em mãos.

In [3]:
df.ctypes.print(); // printa os tipos de dados de cada coluna

╔═════════════════════════╤═════════╗
║ Survived                │ int32   ║
╟─────────────────────────┼─────────╢
║ Pclass                  │ int32   ║
╟─────────────────────────┼─────────╢
║ Name                    │ string  ║
╟─────────────────────────┼─────────╢
║ Sex                     │ string  ║
╟─────────────────────────┼─────────╢
║ Age                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Siblings/Spouses Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Parents/Children Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Fare                    │ float32 ║
╚═════════════════════════╧═════════╝



Vemos que temos duas colunas que são do tipo `string` ('Name' e 'Sex'). 

Para nosso modelo, precisamos que todas as colunas sejam numéricas. As redes neurais que vamos utilizar não conseguem lidar com strings muito bem.

Mas antes de resolver esse problema, vamos ter um panorama estatístico dos dados.

In [4]:
df.describe().print(); // printa estatísticas descritivas do dataframe

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ count      │ 887               │ 887               │ 887               │ 887               │ 887               │ 887               ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ mean       │ 0.3855693348365…  │ 2.3055242390078…  │ 29.471443066516…  │ 0.5253664036076…  │ 0.3833145434047…  │ 32.305420180383…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ std        │ 0.4870041177510…  │ 0.8366620036697…  │ 

### Limpeza de dados

Essa coluna 'Name' não é relevante para nosso modelo, então vamos removê-la.

In [5]:
df = df.drop( { columns: [ 'Name' ] } ); // remove a coluna "Name" do dataframe
df.head().print();

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ male              │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ female            │ 38                │ 1                 │ 0                 │ 71.2833           ║
╟────────────┼───────────────────┼───────────────────┼────────────────

Agora vamos atacar a coluna 'Sex'. Vamos transformar essa coluna em uma coluna numérica.

Vamos primeiro investigar que valores ela assume.

In [6]:
const uniqueValues = df[ 'Sex' ].unique();
uniqueValues.print();

╔═══╤════════╗
║ 0 │ male   ║
╟───┼────────╢
║ 1 │ female ║
╚═══╧════════╝



Vamos verificar se há valores nulos na amostra.

In [7]:
const isNA = df.isNa(); // verifica se há valores NA, associando true a valores NA e false a valores não NA
const isThereNA = isNA.values.some( ( value ) => value === true ); // verifica se há algum valor NA
console.log( isThereNA );

false


Legal. Agora sabemos que não há valores inválidos em toda a amostra e que a coluna 'Sex' só assume os valores 'male' ou 'female'.

Então vamos transformar esses valores em 0 (male) e 1 (female).

In [8]:
df[ 'Sex' ] = df[ 'Sex' ].map( value => value === 'male' ? 0 : 1 );
df.head().print();

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ 0                 │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ 1                 │ 38                │ 1                 │ 0                 │ 71.2833           ║
╟────────────┼───────────────────┼───────────────────┼────────────────

Vamos checar novamente os tipos das colunas.

In [9]:
df.ctypes.print();

╔═════════════════════════╤═════════╗
║ Survived                │ int32   ║
╟─────────────────────────┼─────────╢
║ Pclass                  │ int32   ║
╟─────────────────────────┼─────────╢
║ Sex                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Age                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Siblings/Spouses Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Parents/Children Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Fare                    │ float32 ║
╚═════════════════════════╧═════════╝



Legal. Apenas colunas numéricas. Podemos prosseguir com nossa análise.

### Ajustes na amostra

Se queremos prever a change de um adulto morrer caso estivesse no Titanic, precisamos remover os dados de crianças para adequar a amostra. 

In [10]:
const MAIORIDADE = 18;

let dfMaioridade = df.query( df[ 'Age' ].ge( MAIORIDADE ) ); // aplica um filtro e cria um novo dataframe com os dados filtrados

dfMaioridade.describe().print(); // verifica se o mínimo da idade ficou em 18

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ count      │ 757               │ 757               │ 757               │ 757               │ 757               │ 757               │ 757               ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ mean       │ 0.3659180977542…  │ 2.2483487450462…  │ 0.3315719947159…  │ 32.953104359313…  │ 0.3315719947159…  │ 0.2694848084544…  │ 32.519670673711…  ║
╟────────────┼───────────────────┼───────────────────┼────────────────

Perfeito. Agora temos somente dados de pessoas maiores de idade.
